## Snetence similartiy using Doc2vec

In [2]:
import pandas as pd
import numpy as np
import nltk

In [15]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score


In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/haripillai/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
!pip install gensim

    100% |████████████████████████████████| 24.2MB 247kB/s 
    100% |████████████████████████████████| 71kB 662kB/s 
  Stored in directory: /home/haripillai/.cache/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built smart-open


In [43]:
df=pd.read_csv("../AI?ML/questions.csv")

/home/haripillai/yes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [45]:
df[df.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105799,105796,209841,209842,How can I develop android app?,NaN,0
201877,201871,398348,398349,How can I create an Android app?,NaN,0
363426,363416,711434,711435,NaN,My Chinese name is Haichao Yu. What English na...,0


In [46]:
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)

In [47]:
def remove_stop_word(text):
    stops = set(stopwords.words("english"))
    words = [w for w in text.lower().split() if not w in stops]
    
    final_text = " ".join(words)
    return final_text

In [24]:
df_temp=df


In [48]:
df_temp['q1']=df_temp['question1'].apply(lambda x:remove_stop_word(x))

In [49]:
df_temp['q2']=df_temp['question2'].apply(lambda x:remove_stop_word(x))

In [50]:
df_temp.head(5)

,id,qid1,qid2,question1,question2,is_duplicate,temp,temp1,q1,q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india?,step step guide invest share market india,step step guide invest share market india?,step step guide invest share market?
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor (koh-i-noor) diamond?,story kohinoor ( koh i noor ) diamond,story kohinoor (koh-i-noor) diamond?,would happen indian government stole kohinoor ...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn?,increase speed internet connection using vpn,increase speed internet connection using vpn?,internet speed increased hacking dns?
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely? solve it?,mentally lonely solve it,mentally lonely? solve it?,find remainder [math]23^{24}[/math] divided 24...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"one dissolve water quikly sugar, salt, methane...",one dissolve water quikly sugar salt methane c...,"one dissolve water quikly sugar, salt, methane...",fish would survive salt water?


In [51]:
def remove_special_character(final_text):
        
      final_text = re.sub(r"[^A-Za-z0-9(),!.?\'`]", " ", final_text )
      final_text = re.sub(r"\'s", " 's ", final_text )
      final_text = re.sub(r"\'ve", " 've ", final_text )
      final_text = re.sub(r"n\'t", " 't ", final_text )
      final_text = re.sub(r"\'re", " 're ", final_text )
      final_text = re.sub(r"\'d", " 'd ", final_text )
      final_text = re.sub(r"\'ll", " 'll ", final_text )
      final_text = re.sub(r",", " ", final_text )
      final_text = re.sub(r"\.", " ", final_text )
      final_text = re.sub(r"!", " ", final_text )
      final_text = re.sub(r"\(", " ( ", final_text )
      final_text = re.sub(r"\)", " ) ", final_text )
      final_text = re.sub(r"\?", " ", final_text )
      final_text = re.sub(r"\s{2,}", " ", final_text )
      return final_text

In [52]:
df_temp['q1']=df_temp['q1'].apply(lambda x:remove_special_character(x))

In [53]:
df_temp['q2']=df_temp['q2'].apply(lambda x:remove_special_character(x))

In [42]:
df[df.index==1]
df.loc[1, "question1"]

'What is the story of Kohinoor (Koh-i-Noor) Diamond?'

In [55]:
df_temp.shape

(404359, 10)

In [60]:
question1=list(df_temp['q1'])
question2=list(df_temp['q2'])
qid1=list(df_temp['qid1'])
qid2=list(df_temp['qid2'])


In [62]:
len(qid2)

404359

In [68]:
labeled_questions=[]
for i in range(len(question1)) :
    labeled_questions.append(TaggedDocument(question1[i].split(),str(qid1[i])))
    labeled_questions.append(TaggedDocument(question2[i].split(),str(qid2[i])))
    

In [ ]:
labeled_questions

In [70]:

model = Doc2Vec(dm = 1, min_count=1, window=10, size=150, sample=1e-4, negative=10)
model.build_vocab(labeled_questions)

In [71]:
for epoch in range(20):
    model.train(labeled_questions,epochs=model.iter,total_examples=model.corpus_count)
    print("Epoch #{} is complete.".format(epoch+1))

/home/haripillai/yes/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Epoch #1 is complete.
Epoch #2 is complete.
Epoch #3 is complete.
Epoch #4 is complete.
Epoch #5 is complete.


KeyboardInterrupt: 

In [ ]:
model.most_similar('washington')

In [ ]:
score = model.n_similarity(questions1_split[i],questions2_split[i])